<a href="https://colab.research.google.com/github/raiadi96/Pytorch/blob/master/ResNetModelScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import keras

In [17]:
import keras.backend as K
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers.merge import  add
from keras.models import Model, Sequential, load_model
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [18]:
CHANNEL_AXIS = 3

In [49]:
def identity_block(X, f, filters, stage, block):
  conv_name_base = 'res' + str(stage)+ block+ '_branch'
  bn_name_base = "bn" + str(stage) + block +'_branch'
  F1, F2, F3 = filters
  #Shortcut Filter
  X_shortcut = X

  X = Conv2D(filters = F1, kernel_size = (1,1), strides = (1,1), padding = 'valid',name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base+'2a')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = F2,  kernel_size = (f,f), strides = (1,1), padding = 'same', kernel_initializer = glorot_uniform, name = conv_name_base + '2b')(X)
  X = BatchNormalization(axis = 3, name = bn_name_base+'2b')(X)
  X = Activation('relu')(X)

  X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  X = Add()([X_shortcut, X])
  X = Activation('relu')(X)

  return X
  

In [53]:
def convolutional_block(X, f, filters, stage, block, s = 2):
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  F1, F2, F3 = filters

  X_shortcut = X

  X = Conv2D(F1, (1,1), strides = (s,s), name = conv_name_base + '2a', padding = 'valid', kernel_initializer = glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis = 3,  name = bn_name_base + '2a')(X)
  X = Activation('relu')(X)
  
  X = Conv2D(F2, (f, f), strides = (1,1), name = conv_name_base + '2b', padding = 'same', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
  X = Activation('relu')(X)

  X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)

  return X

In [58]:
def ResNet50(input_shape = (32,32,3), classes = 14):
  X_input = Input(input_shape)

  X =  ZeroPadding2D((3,3))(X_input)

  X = Conv2D(32, (7,7), strides = (1,1), name = 'conv1', kernel_initializer = glorot_uniform(seed = 0))(X)

  X = BatchNormalization(axis = 3, name = 'bn_conv')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3,3))(X)

  X = convolution_block(X, f = 3, filters = [32,32,128], stage = 2, block = 'a', s = 1)
  X = identity_block(X, 3, [32,32,128], stage = 2, block = 'b')
  X = identity_block(X, 3, [32,32,128], stage = 2, block = 'c')

    # Stage 3
  X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 3, block='a', s = 2)
  X = identity_block(X, 3, [64, 64, 256], stage=3, block='b')
  X = identity_block(X, 3, [64, 64, 256], stage=3, block='c')
  X = identity_block(X, 3, [64, 64, 256], stage=3, block='d')

    # Stage 4 
  X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 4, block='a', s = 2)
  X = identity_block(X, 3, [128, 128, 512], stage=4, block='b')
  X = identity_block(X, 3, [128, 128, 512], stage=4, block='c')
  X = identity_block(X, 3, [128, 128, 512], stage=4, block='d')
  X = identity_block(X, 3, [128, 128, 512], stage=4, block='e')
  X = identity_block(X, 3, [128, 128, 512], stage=4, block='f')

    # Stage 5 
  X = convolutional_block(X, f = 3, filters = [256,256, 1024], stage = 5, block='a', s = 2)
  X = identity_block(X, 3, [256,256, 1024], stage=5, block='b')
  X = identity_block(X, 3, [256,256, 1024], stage=5, block='c')

  X = AveragePooling2D(pool_size = (2,2), name = 'avg_pool')(X)

  X = Flatten()(X)
  X = Dense(classes, activation = 'softmax', name = 'fc'+str(classes), kernel_initializer = glorot_uniform(seed = 0))(X)

  model = Model(inputs = X_input, outputs = X, name = 'Resnet50')

  return model

In [59]:
model = ResNet50(input_shape = (32,32,3), classes= 14)

In [60]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [61]:
model.summary()

Model: "Resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_11 (ZeroPadding2 (None, 38, 38, 3)    0           input_15[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 32)   4736        zero_padding2d_11[0][0]          
__________________________________________________________________________________________________
bn_conv (BatchNormalization)    (None, 32, 32, 32)   128         conv1[0][0]                      
___________________________________________________________________________________________